# 通过深度卷积神经网络分类图片

- 理解在1到2个维度上的卷积操作
- 了解CNN架构的构建模块
- 用TensorFlow实现深度卷积神经网络

## 卷积神经网络的构建模块

### 了解CNN和学习功能层次结构

- Convolutional layers
- Pooling layers
- Fully Connected layers

### 执行离散卷积

下面出现的$*$是指的卷积操作,而不是乘法

#### 执行1维下的离散卷积

\begin{equation}
\boldsymbol{y}=\boldsymbol{x} * \boldsymbol{w} \rightarrow \boldsymbol{y}[i]=\sum_{k=-\infty}^{+\infty} \boldsymbol{x}[i-k] \boldsymbol{w}[k]
\end{equation}

假设x和w有n和m个元素
\begin{equation}
y = x * w \rightarrow y[i] = \sum_{k=0}^{k=m-1}x^p[i+m-k]w[k]
\end{equation}

![1.png](1.png)

#### 零填充在卷积中的效果

The following figure illustrates the three different padding modes for a simple 5 x 5
pixel input with a kernel size of 3 x 3 and a stride of 1
![2](2.png)

#### 决定卷积输出的大小

\begin{equation}
O=\left\lfloor\frac{n+2 p-m}{s}\right\rfloor+ 1
\end{equation}

n是x的大小, m是kernel的大小, p是padding, s是stride

- same model
\begin{equation}
n=10, m=5, p=2, s=1 \rightarrow o=\left\lfloor\frac{10+2 \times 2-5}{1}\right\rfloor+ 1=10
\end{equation}

In [1]:
import numpy as np


def conv1d(x, w, p=0, s=1):
    w_rot = np.array(w[::-1])
    x_padded = np.array(x)
    if p > 0:
        zero_pad = np.zeros(shape=p)
        x_padded = np.concatenate([zero_pad, x_padded, zero_pad])

    res = []
    for i in range(0, int(len(x)/s), s):
        res.append(np.sum(x_padded[i:i+w_rot.shape[0]] * w_rot))

    return np.array(res)

In [2]:
# Testing
x = [1, 3, 2, 4, 5, 6, 1, 3]
w = [1, 0, 3, 1, 2]
print('Conv1d Implementation:', conv1d(x, w, p=2, s=1))

Conv1d Implementation: [ 5. 14. 16. 26. 24. 34. 19. 22.]


In [3]:
print('Numpy Results:', np.convolve(x, w, mode='same'))

Numpy Results: [ 5 14 16 26 24 34 19 22]


#### 执行2维下的离散卷积

\begin{equation}
\boldsymbol{Y}=\boldsymbol{X} * \boldsymbol{W} \rightarrow \boldsymbol{Y}[i, j]=\sum_{k_{1}=-\infty}^{+\infty} \sum_{k_{2}=-\infty}^{+\infty} \boldsymbol{X}\left[i-k_{1}, j-k_{2}\right] \boldsymbol{W}\left[k_{1}, k_{2}\right]
\end{equation}

![3](3.png)

\begin{equation}
\boldsymbol{W}^{r}=\left[\begin{array}{lll}{0.5} & {1} & {0.5} \\ {0.1} & {0.4} & {0.3} \\ {0.4} & {0.7} & {0.5}\end{array}\right]
\end{equation}

p=(1,1), s=(2,2)

![4](4.png)

In [4]:
import numpy as np
import scipy.signal


def conv2d(X, W, p=(0, 0), s=(1, 1)):
    W_rot = np.array(W)[::-1, ::-1]
    X_orig = np.array(X)
    n1 = X_orig.shape[0] + 2*p[0]
    n2 = X_orig.shape[1] + 2*p[1]
    X_padded = np.zeros(shape=(n1, n2))
    X_padded[p[0]:p[0]+X_orig.shape[0], p[1]:p[1]+X_orig.shape[1]] = X_orig

    res = []
    for i in range(0, int((X_padded.shape[0] - W_rot.shape[0])/s[0])+1, s[0]):
        res.append([])
        for j in range(0, int((X_padded.shape[1] - W_rot.shape[1])/s[1])+1, s[1]):
            X_sub = X_padded[i:i+W_rot.shape[0], j:j+W_rot.shape[1]]
            res[-1].append(np.sum(X_sub*W_rot))
    return (np.array(res))

In [5]:
X = [[1, 3, 2, 4], [5, 6, 1, 3], [1, 2, 0, 2], [3, 4, 3, 2]]
W = [[1, 0, 3], [1, 2, 1], [0, 1, 1]]

print('Conv2d Implementation:\n', conv2d(X, W, p=(1, 1), s=(1, 1)))

Conv2d Implementation:
 [[11. 25. 32. 13.]
 [19. 25. 24. 13.]
 [13. 28. 25. 17.]
 [11. 17. 14.  9.]]


In [6]:
print('Scipy Results:\n', scipy.signal.convolve2d(X, W, mode='same'))

Scipy Results:
 [[11 25 32 13]
 [19 25 24 13]
 [13 28 25 17]
 [11 17 14  9]]


### 子采样

![5](5.png)

优点有两点:
1. Pooling(max-pooling)有更好的robust性能
2. 减少了特征数量, 增加了计算效率并同时降低了过拟合的程度

## 融合所有东西构建一个CNN

### 使用多个输入或颜色通道

![6](6.png)

若有多个feature map,则变化如下
![7](7.png)

卷积层后接一个采样层的示例图
![8](8.png)

### 用dropout正则化神经网络

![9](9.png)

## 使用TensorFlow实现一个深度卷积神经网络

### 多层的CNN架构

![10](10.png)

- Input: $[batchsize \times 28 \times 28 \times 1]$
- Conv_1: $[batchsize \times 24 \times 24 \times 32]$
- Pooling_1: $[batchsize \times 12 \times 12 \times 32]$
- Conv_2: $[batchsize \times 8 \times 8 \times 64]$
- Pooling_2: $[batchsize \times 4 \times 4 \times 64]$
- FC_1: $[batchsize \times 1024]$
- FC_2 and softmax layer: $[batchsize \times 10]$

### 加载和预处理数据

In [7]:
import os
import struct
import numpy as np

def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path, '{}-labels-idx1-ubyte'.format(kind))
    images_path = os.path.join(path, '{}-images-idx3-ubyte'.format(kind))

    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', lbpath.read(8))
        labels = np.fromfile(lbpath, dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack('>IIII', imgpath.read(16))
        images = np.fromfile(imgpath, dtype=np.uint8).reshape(len(labels), 784)

    return images, labels

In [8]:
# loading the data
X_data, y_data = load_mnist('./mnist/', kind='train')
print('Rows: {}, Columns: {}'.format(X_data.shape[0], X_data.shape[1]))
X_test, y_test = load_mnist('./mnist/', kind='t10k')
print('Rows: {}, Columns: {}'.format(X_test.shape[0], X_test.shape[1]))
X_train, y_train = X_data[:50000, :], y_data[:50000]
X_valid, y_valid = X_data[50000:, :], y_data[50000:]

print('Training:   ', X_train.shape, y_train.shape)
print('Validation: ', X_valid.shape, y_valid.shape)
print('Test Set:   ', X_test.shape, y_test.shape)

Rows: 60000, Columns: 784
Rows: 10000, Columns: 784
Training:    (50000, 784) (50000,)
Validation:  (10000, 784) (10000,)
Test Set:    (10000, 784) (10000,)


In [9]:
def batch_generator(X, y, batch_size=64, shuffle=False, random_seed=None):
    idx = np.arange(y.shape[0])
    
    if shuffle:
        rng = np.random.RandomState(random_seed)
        rng.shuffle(idx)
        X = X[idx]
        y = y[idx]
    
    for i in range(0, X.shape[0], batch_size):
        yield (X[i:i+batch_size, :], y[i:i+batch_size])

In [10]:
mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)

X_train_centered = (X_train - mean_vals)/std_val
X_valid_centered = (X_valid - mean_vals)/std_val
X_test_centered = (X_test - mean_vals)/std_val

### 使用TensorFlow 的低级API实现一个CNN

In [11]:
import tensorflow as tf
import numpy as np

def conv_layer(input_tensor, name, kernel_size, n_output_channels, padding_mode='SAME', strides=(1, 1, 1, 1)):
    with tf.variable_scope(name):
        input_shape = input_tensor.get_shape().as_list()
        n_input_channels = input_shape[-1]
        
        weights_shape = list(kernel_size) + [n_input_channels, n_output_channels]
        weights = tf.get_variable(name='_weights', shape=weights_shape)
        print(weights)
        
        biases = tf.get_variable(name='_biases', initializer=tf.zeros(shape=[n_output_channels]))
        print(biases)
        
        conv = tf.nn.conv2d(input=input_tensor, filter=weights, strides=strides, padding=padding_mode)
        print(conv)
        
        conv = tf.nn.bias_add(conv, biases, name='net_pre-activation')
        print(conv)
        
        conv = tf.nn.relu(conv, name='activation')
        print(conv)
    
        return conv   

In [12]:
g = tf.Graph()
with g.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
    conv_layer(x, name='convtest', kernel_size=(3, 3), n_output_channels=32)
del g, x

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
<tf.Variable 'convtest/_weights:0' shape=(3, 3, 1, 32) dtype=float32_ref>
<tf.Variable 'convtest/_biases:0' shape=(32,) dtype=float32_ref>
Tensor("convtest/Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("convtest/net_pre-activation:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("convtest/activation:0", shape=(?, 28, 28, 32), dtype=float32)


In [13]:
def fc_layer(input_tensor, name, n_output_units, activation_fn=None):
    with tf.variable_scope(name):
        input_shape = input_tensor.get_shape().as_list()[1:]
        n_input_units = np.prod(input_shape)
        if len(input_shape)>1:
            input_tensor = tf.reshape(input_tensor, shape=(-1, n_input_units))
        
        weight_shape = (n_input_units, n_output_units)
        weights = tf.get_variable(name='_weights', shape=weight_shape)
        print(weights)
        biases = tf.get_variable(name='_biases', initializer=tf.zeros(shape=[n_output_units]))
        print(biases)
        layer = tf.matmul(input_tensor, weights)
        print(layer)
        layer = tf.nn.bias_add(layer, biases, name='net_pre-activation')
        print(layer)
        if activation_fn is None:
            return layer
        layer = activation_fn(layer, name='activation')
        print(layer)
        return layer

In [14]:
g = tf.Graph()
with g.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
    fc_layer(x, name='fctest', n_output_units=32, activation_fn=tf.nn.relu)

del g, x

<tf.Variable 'fctest/_weights:0' shape=(784, 32) dtype=float32_ref>
<tf.Variable 'fctest/_biases:0' shape=(32,) dtype=float32_ref>
Tensor("fctest/MatMul:0", shape=(?, 32), dtype=float32)
Tensor("fctest/net_pre-activation:0", shape=(?, 32), dtype=float32)
Tensor("fctest/activation:0", shape=(?, 32), dtype=float32)


In [28]:
def build_cnn(learning_rate=1e-4):
    ## Placeholders for X and y:
    tf_x = tf.placeholder(tf.float32, shape=[None, 784],
                          name='tf_x')
    tf_y = tf.placeholder(tf.int32, shape=[None],
                          name='tf_y')

    # reshape x to a 4D tensor: 
    # [batchsize, width, height, 1]
    tf_x_image = tf.reshape(tf_x, shape=[-1, 28, 28, 1],
                            name='tf_x_reshaped')
    ## One-hot encoding:
    tf_y_onehot = tf.one_hot(indices=tf_y, depth=10,
                             dtype=tf.float32,
                             name='tf_y_onehot')

    ## 1st layer: Conv_1
    print('\nBuilding 1st layer: ')
    h1 = conv_layer(tf_x_image, name='conv_1',
                    kernel_size=(5, 5), 
                    padding_mode='VALID',
                    n_output_channels=32)
    ## MaxPooling
    h1_pool = tf.nn.max_pool(h1, 
                             ksize=[1, 2, 2, 1],
                             strides=[1, 2, 2, 1], 
                             padding='SAME')
    ## 2n layer: Conv_2
    print('\nBuilding 2nd layer: ')
    h2 = conv_layer(h1_pool, name='conv_2', 
                    kernel_size=(5,5), 
                    padding_mode='VALID',
                    n_output_channels=64)
    ## MaxPooling 
    h2_pool = tf.nn.max_pool(h2, 
                             ksize=[1, 2, 2, 1],
                             strides=[1, 2, 2, 1], 
                             padding='SAME')

    ## 3rd layer: Fully Connected
    print('\nBuilding 3rd layer:')
    h3 = fc_layer(h2_pool, name='fc_3',
                  n_output_units=1024, 
                  activation_fn=tf.nn.relu)

    ## Dropout
    keep_prob = tf.placeholder(tf.float32, name='fc_keep_prob')
    h3_drop = tf.nn.dropout(h3, keep_prob=keep_prob, 
                            name='dropout_layer')

    ## 4th layer: Fully Connected (linear activation)
    print('\nBuilding 4th layer:')
    h4 = fc_layer(h3_drop, name='fc_4',
                  n_output_units=10, 
                  activation_fn=None)

    ## Prediction
    predictions = {
        'probabilities' : tf.nn.softmax(h4, name='probabilities'),
        'labels' : tf.cast(tf.argmax(h4, axis=1), tf.int32,
                           name='labels')
    }
    
    ## Visualize the graph with TensorBoard:

    ## Loss Function and Optimization
    cross_entropy_loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            logits=h4, labels=tf_y_onehot),
        name='cross_entropy_loss')

    ## Optimizer:
    optimizer = tf.train.AdamOptimizer(learning_rate)
    optimizer = optimizer.minimize(cross_entropy_loss,
                                   name='train_op')

    ## Computing the prediction accuracy
    correct_predictions = tf.equal(
        predictions['labels'], 
        tf_y, name='correct_preds')

    accuracy = tf.reduce_mean(
        tf.cast(correct_predictions, tf.float32),
        name='accuracy')

![11](11.png)

In [29]:
def save(saver, sess, epoch, path='./model/'):
    if not os.path.isdir(path):
        os.makedirs(path)
    print('Saving model in %s' % path)
    saver.save(sess, os.path.join(path,'cnn-model.ckpt'),
               global_step=epoch)

    
def load(saver, sess, path, epoch):
    print('Loading model from %s' % path)
    saver.restore(sess, os.path.join(
            path, 'cnn-model.ckpt-%d' % epoch))

    
def train(sess, training_set, validation_set=None,
          initialize=True, epochs=20, shuffle=True,
          dropout=0.5, random_seed=None):

    X_data = np.array(training_set[0])
    y_data = np.array(training_set[1])
    training_loss = []

    ## initialize variables
    if initialize:
        sess.run(tf.global_variables_initializer())

    np.random.seed(random_seed) # for shuflling in batch_generator
    for epoch in range(1, epochs+1):
        batch_gen = batch_generator(
                        X_data, y_data, 
                        shuffle=shuffle)
        avg_loss = 0.0
        for i,(batch_x,batch_y) in enumerate(batch_gen):
            feed = {'tf_x:0': batch_x, 
                    'tf_y:0': batch_y, 
                    'fc_keep_prob:0': dropout}
            loss, _ = sess.run(
                    ['cross_entropy_loss:0', 'train_op'],
                    feed_dict=feed)
            avg_loss += loss

        training_loss.append(avg_loss / (i+1))
        print('Epoch %02d Training Avg. Loss: %7.3f' % (
            epoch, avg_loss), end=' ')
        if validation_set is not None:
            feed = {'tf_x:0': validation_set[0],
                    'tf_y:0': validation_set[1],
                    'fc_keep_prob:0':1.0}
            valid_acc = sess.run('accuracy:0', feed_dict=feed)
            print(' Validation Acc: %7.3f' % valid_acc)
        else:
            print()

            
def predict(sess, X_test, return_proba=False):
    feed = {'tf_x:0': X_test, 
            'fc_keep_prob:0': 1.0}
    if return_proba:
        return sess.run('probabilities:0', feed_dict=feed)
    else:
        return sess.run('labels:0', feed_dict=feed)

In [30]:
# Define hyperparameters
learning_rate = 1e-4
random_seed = 123

# create a graph
g = tf.Graph()
with g.as_default():
    tf.set_random_seed(random_seed)
    # build the graph
    build_cnn()
    # saver
    saver = tf.train.Saver()


Building 1st layer: 
<tf.Variable 'conv_1/_weights:0' shape=(5, 5, 1, 32) dtype=float32_ref>
<tf.Variable 'conv_1/_biases:0' shape=(32,) dtype=float32_ref>
Tensor("conv_1/Conv2D:0", shape=(?, 24, 24, 32), dtype=float32)
Tensor("conv_1/net_pre-activation:0", shape=(?, 24, 24, 32), dtype=float32)
Tensor("conv_1/activation:0", shape=(?, 24, 24, 32), dtype=float32)

Building 2nd layer: 
<tf.Variable 'conv_2/_weights:0' shape=(5, 5, 32, 64) dtype=float32_ref>
<tf.Variable 'conv_2/_biases:0' shape=(64,) dtype=float32_ref>
Tensor("conv_2/Conv2D:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("conv_2/net_pre-activation:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("conv_2/activation:0", shape=(?, 8, 8, 64), dtype=float32)

Building 3rd layer:
<tf.Variable 'fc_3/_weights:0' shape=(1024, 1024) dtype=float32_ref>
<tf.Variable 'fc_3/_biases:0' shape=(1024,) dtype=float32_ref>
Tensor("fc_3/MatMul:0", shape=(?, 1024), dtype=float32)
Tensor("fc_3/net_pre-activation:0", shape=(?, 1024), dtype=float32

In [31]:
# create a TF session
# and train the model

with tf.Session(graph=g) as sess:
    train(sess, training_set=(X_train_centered, y_train), validation_set=(X_valid_centered, y_valid),
         initialize=True, random_seed=123)
    save(saver, sess, epoch=20)

Epoch 01 Training Avg. Loss: 274.095  Validation Acc:   0.973
Epoch 02 Training Avg. Loss:  75.237  Validation Acc:   0.982
Epoch 03 Training Avg. Loss:  51.263  Validation Acc:   0.987
Epoch 04 Training Avg. Loss:  38.751  Validation Acc:   0.988
Epoch 05 Training Avg. Loss:  32.076  Validation Acc:   0.989
Epoch 06 Training Avg. Loss:  28.159  Validation Acc:   0.988
Epoch 07 Training Avg. Loss:  22.958  Validation Acc:   0.988
Epoch 08 Training Avg. Loss:  19.480  Validation Acc:   0.990
Epoch 09 Training Avg. Loss:  17.448  Validation Acc:   0.991
Epoch 10 Training Avg. Loss:  15.199  Validation Acc:   0.992
Epoch 11 Training Avg. Loss:  13.287  Validation Acc:   0.990
Epoch 12 Training Avg. Loss:  11.767  Validation Acc:   0.992
Epoch 13 Training Avg. Loss:  10.374  Validation Acc:   0.990
Epoch 14 Training Avg. Loss:   8.826  Validation Acc:   0.992
Epoch 15 Training Avg. Loss:   8.516  Validation Acc:   0.992
Epoch 16 Training Avg. Loss:   7.834  Validation Acc:   0.992
Epoch 17

In [32]:
# Calculate prediction accuracy
# on test set
# restoring the saved model

del g

# create a new graph
# and bulild the model
g2 = tf.Graph()
with g2.as_default():
    tf.set_random_seed(random_seed)
    # build the graph
    build_cnn()
    # saver
    saver = tf.train.Saver()
    
# create a new session
# and restore the model
with tf.Session(graph=g2) as sess:
    load(saver, sess, epoch=20, path='./model/')
    
    preds = predict(sess, X_test_centered, return_proba=False)
    print('Test Accuracy: {:.3%}'.format(np.sum(preds==y_test)/len(y_test)))


Building 1st layer: 
<tf.Variable 'conv_1/_weights:0' shape=(5, 5, 1, 32) dtype=float32_ref>
<tf.Variable 'conv_1/_biases:0' shape=(32,) dtype=float32_ref>
Tensor("conv_1/Conv2D:0", shape=(?, 24, 24, 32), dtype=float32)
Tensor("conv_1/net_pre-activation:0", shape=(?, 24, 24, 32), dtype=float32)
Tensor("conv_1/activation:0", shape=(?, 24, 24, 32), dtype=float32)

Building 2nd layer: 
<tf.Variable 'conv_2/_weights:0' shape=(5, 5, 32, 64) dtype=float32_ref>
<tf.Variable 'conv_2/_biases:0' shape=(64,) dtype=float32_ref>
Tensor("conv_2/Conv2D:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("conv_2/net_pre-activation:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("conv_2/activation:0", shape=(?, 8, 8, 64), dtype=float32)

Building 3rd layer:
<tf.Variable 'fc_3/_weights:0' shape=(1024, 1024) dtype=float32_ref>
<tf.Variable 'fc_3/_biases:0' shape=(1024,) dtype=float32_ref>
Tensor("fc_3/MatMul:0", shape=(?, 1024), dtype=float32)
Tensor("fc_3/net_pre-activation:0", shape=(?, 1024), dtype=float32

In [33]:
# run the prediction on som test samples
np.set_printoptions(precision=2, suppress=True)

with tf.Session(graph=g2) as sess:
    load(saver, sess, epoch=20, path='./model/')
    
    print(predict(sess, X_test_centered[:10], return_proba=False))
    print(predict(sess, X_test_centered[:10], return_proba=True))

Loading model from ./model/
INFO:tensorflow:Restoring parameters from ./model/cnn-model.ckpt-20
[7 2 1 0 4 1 4 9 5 9]
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [34]:
# continue training for 20 more epochs
# without re-initializing :: initialize=False
# create a new session
# and restore the model
with tf.Session(graph=g2) as sess:
    load(saver, sess, epoch=20, path='./model/')
    train(sess, training_set=(X_train_centered, y_train),
         validation_set=(X_valid_centered, y_valid), initialize=False, epochs=20, random_seed=123)
    save(saver, sess, epoch=40, path='./model/')
    preds = predict(sess, X_test_centered, return_proba=False)
    print('Test Accuracy: {:.3%}'.format(np.sum(preds == y_test)/len(y_test)))

Loading model from ./model/
INFO:tensorflow:Restoring parameters from ./model/cnn-model.ckpt-20
Epoch 01 Training Avg. Loss:   4.373  Validation Acc:   0.991
Epoch 02 Training Avg. Loss:   4.315  Validation Acc:   0.991
Epoch 03 Training Avg. Loss:   3.540  Validation Acc:   0.993
Epoch 04 Training Avg. Loss:   3.805  Validation Acc:   0.993
Epoch 05 Training Avg. Loss:   2.762  Validation Acc:   0.992
Epoch 06 Training Avg. Loss:   3.653  Validation Acc:   0.993
Epoch 07 Training Avg. Loss:   2.355  Validation Acc:   0.993
Epoch 08 Training Avg. Loss:   2.734  Validation Acc:   0.992
Epoch 09 Training Avg. Loss:   2.319  Validation Acc:   0.992
Epoch 10 Training Avg. Loss:   2.076  Validation Acc:   0.992
Epoch 11 Training Avg. Loss:   2.375  Validation Acc:   0.992
Epoch 12 Training Avg. Loss:   1.592  Validation Acc:   0.992
Epoch 13 Training Avg. Loss:   2.486  Validation Acc:   0.992
Epoch 14 Training Avg. Loss:   2.036  Validation Acc:   0.993
Epoch 15 Training Avg. Loss:   1.682

### 使用TensorFlow的Layers API来实现CNN

In [37]:
import tensorflow as tf
import numpy as np


class ConvNN(object):
    def __init__(self, batchsize=64, epochs=20, learning_rate=1e-4,
                 dropout_rate=0.5, shuffle=True, random_seed=None):
        np.random.seed(random_seed)
        self.batchsize = batchsize
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.dropout_rate = dropout_rate
        self.shuffle = shuffle

        g = tf.Graph()
        with g.as_default():
            # set random seed
            tf.set_random_seed(random_seed)
            # build the network
            self.build()
            # initializer
            self.init_op = tf.global_variables_initializer()
            # saver
            self.saver = tf.train.Saver()

        # create a session
        self.sess = tf.Session(graph=g)

    def build(self):
        # Placeholders
        tf_x = tf.placeholder(tf.float32, shape=[None, 784], name='tf_x')
        tf_y = tf.placeholder(tf.int32, shape=[None], name='tf_y')
        is_train = tf.placeholder(tf.bool, shape=(), name='is_train')

        # reshape x to a 4D tensor
        # [batchsize, width, height, 1]
        tf_x_image = tf.reshape(
            tf_x, shape=[-1, 28, 28, 1], name='input_x_2dimages')

        # one-hot encoding
        tf_y_onehot = tf.one_hot(
            indices=tf_y, depth=10, dtype=tf.float32, name='input_y_onehot')

        # 1st layer:Conv_1
        h1 = tf.layers.conv2d(tf_x_image, kernel_size=(
            5, 5), filters=32, activation=tf.nn.relu)
        # Maxpooling
        h1_pool = tf.layers.max_pooling2d(h1, pool_size=(2, 2), strides=(2, 2))

        # 2nd layer:Conv_2
        h2 = tf.layers.conv2d(h1_pool, kernel_size=(
            5, 5), filters=64, activation=tf.nn.relu)
        # Maxpooling
        h2_pool = tf.layers.max_pooling2d(h2, pool_size=(2, 2), strides=(2, 2))

        # 3rd layer: fc
        input_shape = h2_pool.get_shape().as_list()
        n_input_units = np.prod(input_shape[1:])
        h2_pool_flat = tf.reshape(h2_pool, shape=[-1, n_input_units])
        h3 = tf.layers.dense(h2_pool_flat, 1024, activation=tf.nn.relu)
        # dropout
        h3_drop = tf.layers.dropout(
            h3, rate=self.dropout_rate, training=is_train)

        # 4th layer: fc(linear activation)
        h4 = tf.layers.dense(h3_drop, 10, activation=None)

        # prediction
        predictions = {
            'probabilities': tf.nn.softmax(h4, name='probabilities'),
            'labels': tf.cast(tf.argmax(h4, axis=1), tf.int32, name='labels')
        }

        # Loss Function and Optimization
        cross_entropy_loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(logits=h4, labels=tf_y_onehot), name='cross_entropy_loss')

        # Optimizer
        optimizer = tf.train.AdamOptimizer(self.learning_rate)
        optimizer = optimizer.minimize(cross_entropy_loss, name='train_op')

        # Finding accuracy
        correct_predictions = tf.equal(
            predictions['labels'], tf_y, name='correct_preds')

        accuracy = tf.reduce_mean(
            tf.cast(correct_predictions, tf.float32), name='accuracy')

    def save(self, epoch, path='./tflayers-model/'):
        if not os.path.isdir(path):
            os.makedirs(path)
        print('Saving model in {}'.format(path))
        self.saver.save(self.sess, os.path.join(
            path, 'model.ckpt'), global_step=epoch)

    def load(self, epoch, path):
        print('Loading model from {}'.format(path))
        self.saver.restore(self.sess, os.path.join(
            path, 'model.ckpt-{}'.format(epoch)))

    def train(self, training_set, validation_set=None, initialize=True):
        if initialize:
            self.sess.run(self.init_op)

        self.train_cost_ = []
        X_data = np.array(training_set[0])
        y_data = np.array(training_set[1])

        for epoch in range(1, self.epochs+1):
            batch_gen = batch_generator(X_data, y_data, shuffle=self.shuffle)
            avg_loss = 0.0
            for i, (batch_x, batch_y) in enumerate(batch_gen):
                feed = {'tf_x:0': batch_x,
                        'tf_y:0': batch_y,
                        'is_train:0': True}
                loss, _ = self.sess.run(
                    ['cross_entropy_loss:0', 'train_op'], feed_dict=feed)
                avg_loss += loss
            print('Epoch %02d: Training Avg. Loss: '
                  '%7.3f' % (epoch, avg_loss), end=' ')
            if validation_set is not None:
                feed = {'tf_x:0': batch_x,
                        'tf_y:0': batch_y,
                        'is_train:0': False}  # for dropout
                valid_acc = self.sess.run('accuracy:0',
                                          feed_dict=feed)
                print('Validation Acc: %7.3f' % valid_acc)
            else:
                print()

    def predict(self, X_test, return_proba=False):
        feed = {'tf_x:0': X_test,
                'is_train:0': False}  # for dropout
        if return_proba:
            return self.sess.run('probabilities:0',
                                 feed_dict=feed)
        else:
            return self.sess.run('labels:0',
                                 feed_dict=feed)